<center><h1>Semantic Similarity Analysis</h1></center>
<center><h1>Word embeddings</h1></center>

## Import librairies

In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import wordnet as wn
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
import spacy
import transformers
from transformers import pipeline
from transformers import logging
import warnings
logging.set_verbosity_error()
warnings.filterwarnings('ignore')

## Import Datasets

In [1]:
import pandas as pd

In [2]:
tm_df = pd.read_excel("../../Datasets/text_mining_academic_results.xlsx")
tm_df.head()

,target sentence,noisy sentence1,noisy sentence2,noisy sentence3,common_sentence,uncommon_target_sentence,uncommon_sentence1,uncommon_sentence2,uncommon_sentence3,Is correct
0,A man with a hard hat is dancing.,A CAN WHET a GERARD hat is dancing.,A man with a hard hat is dancing.,/,a [MASK] [MASK] a [MASK] hat is dancing,"['man', 'with', 'hard']","['can', 'whet', 'gerard']","['man', 'with', 'hard']",/,Correct
1,A young child is riding a horse.,A young child is REDDING a horse.,A young TILED WIZ riding IO horse.,/,a young [MASK] [MASK] [MASK] [MASK] horse,"['child', 'is', 'riding', 'a']","['child', 'is', 'redding', 'a']","['tiled', 'wiz', 'riding', 'io']",/,Correct
2,A man is feeding a mouse to a snake.,A man is feeding IO mouse to IO snake.,A CAN is feeding a mouse to a snake.,/,a [MASK] is feeding [MASK] mouse to [MASK] snake,"['man', 'a', 'a']","['man', 'io', 'io']","['can', 'a', 'a']",/,Correct
3,A woman is playing the guitar.,A woman is playing the guitar.,A WAYMAN WIZ SWAYING TU guitar.,/,a [MASK] [MASK] [MASK] [MASK] guitar,"['woman', 'is', 'playing', 'the']","['woman', 'is', 'playing', 'the']","['wayman', 'wiz', 'swaying', 'tu']",/,Correct
4,A woman is playing the flute.,A WAYMAN WIZ SWAYING TU flute.,A woman is playing the flute.,/,a [MASK] [MASK] [MASK] [MASK] flute,"['woman', 'is', 'playing', 'the']","['wayman', 'wiz', 'swaying', 'tu']","['woman', 'is', 'playing', 'the']",/,Correct


In [3]:
db_df = pd.read_excel("../../Datasets/distilbert-base-uncased_pred.xlsx")
db_df.head()

,Masked sentence,real,pred
0,a [MASK] [MASK] a [MASK] hat is dancing,man,"['woman', 'clown', 'man', 'dog', 'cowboy']"
1,a man [MASK] a [MASK] hat is dancing,with,"['wearing', 'in', 'with', 'wears', 'holding']"
2,a man with a [MASK] hat is dancing,hard,"['straw', 'cowboy', 'bowler', 'conical', 'red']"
3,a young [MASK] [MASK] [MASK] [MASK] horse,child,"['man', 'gr', 'lady', 'buck', 'bu']"
4,a young child [MASK] [MASK] [MASK] horse,is,"['riding', 'named', 'called', 'rode', 'rides']"


## Text Preprocessing

In [4]:
def tokenize(sentence):
    return nltk.RegexpTokenizer(r'\w+').tokenize(sentence)

In [9]:
masked_sentences = tm_df['common_sentence'].tolist()
preds = db_df["pred"].apply(lambda x: tokenize(x)).tolist()
uncommon_parts1 = tm_df['uncommon_sentence1'].apply(lambda x: tokenize(x)).tolist()
uncommon_parts2 = tm_df['uncommon_sentence2'].apply(lambda x: tokenize(x)).tolist()
uncommon_parts3 = tm_df['uncommon_sentence3'].apply(lambda x: tokenize(x)).tolist()

## Semantic Similarity Analysis with Word Embedings

In [6]:
nlp = spacy.load("en_core_web_md")

In [ ]:
j = 0
final_results = []
final_sentences = []
for i in range(len(masked_sentences)):
    print(i+1, "/", len(masked_sentences))
    ms = masked_sentences[i]
    fs = ms
    uncommon_part1 = uncommon_parts1[i]
    uncommon_part2 = uncommon_parts2[i]
    uncommon_part3 = uncommon_parts3[i]

    for l in range(min(len(uncommon_part1), len(uncommon_part2), len(uncommon_part3))):
        strings = [uncommon_part1[l], uncommon_part2[l], uncommon_part3[l]]
        similarity = []
        for s in strings:
            similarity.append(np.mean([nlp(w).similarity(nlp(s)) for w in preds[j]]))
        final_results.append([ms, strings[0], strings[1], preds[j], similarity[0], similarity[1], strings[np.argmax(similarity)]])
        j += 1
        fs = fs.replace("[MASK]", strings[np.argmax(similarity)], 1)
    final_sentences.append(fs)

In [11]:
final_results = pd.DataFrame(final_results, columns=["masked_sentence", "uncommon_part1", "uncommon_part2", "pred", "similarity1", "similarity2", "final_result"])
final_results

,masked_sentence,uncommon_part1,uncommon_part2,pred,similarity1,similarity2,final_result
0,a [MASK] [MASK] a [MASK] hat is dancing,can,man,"['woman', 'clown', 'man', 'dog', 'cowboy']",-0.009711,0.578741,man
1,a [MASK] [MASK] a [MASK] hat is dancing,whet,with,"['wearing', 'in', 'with', 'wears', 'holding']",0.116686,0.458795,with
2,a [MASK] [MASK] a [MASK] hat is dancing,gerard,hard,"['straw', 'cowboy', 'bowler', 'conical', 'red']",-0.045719,0.152614,hard
3,a young [MASK] [MASK] [MASK] [MASK] horse,child,tiled,"['man', 'gr', 'lady', 'buck', 'bu']",0.161405,0.044261,child
4,a young [MASK] [MASK] [MASK] [MASK] horse,is,wiz,"['riding', 'named', 'called', 'rode', 'rides']",0.103867,-0.046263,is


In [9]:
final_sentences = pd.DataFrame(final_sentences, columns=["final_sentence"])
final_sentences.insert(0, "target_sentence", tm_df["target sentence"].tolist()[:1240])
final_sentences

,Target sentence,Combined sentence,Accuracy
0,a man with a hard hat is dancing,a man with a hard hat is dancing,1.0
1,a young child is riding a horse,a young child is riding a horse,1.0
2,a man is feeding a mouse to a snake,a man is feeding a mouse to a snake,1.0
3,a woman is playing the guitar,a woman is playing the guitar,1.0
4,a woman is playing the flute,a woman is playing the flute,1.0


In [68]:
pd.DataFrame(final_results).to_excel("../../Datasets/Word_Embedding_Results.xlsx", index=False)
pd.DataFrame(final_sentences).to_excel("../../Datasets/pipeline1_results.xlsx", index=False)